## Import necessary modules
Run this cell before running any other cells

In [1]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [2]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2023-02-04 16:16:00,230 | INFO     |: info
2023-02-04 16:16:00,231 | WARNING  |: warning
2023-02-04 16:16:00,232 | ERROR    |: error
2023-02-04 16:16:00,233 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In [3]:
from uuid import uuid4
for i in range(4):
    print(str(uuid4())+ "\n")
    

e0dccea4-f3ea-43c4-aeda-ae1734294918

a2ed6bad-2c9e-4326-ab63-d354147f4cf1

875f41fa-540b-4c32-a3d5-bd70b9675248

01dbc6ad-0ff8-441a-9011-8618d0fcc0ab



In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [23]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2023-02-04 17:43:46,504 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:0c:66:2f:3c
2023-02-04 17:43:51,635 | INFO     |: Connected to c0:83:0c:66:2f:3c


## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [24]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

5.5


In [25]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

[->9.0<-]


## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [26]:
ble.send_command(CMD.PING, "")

In [27]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

PONG


The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [21]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

2023-02-04 17:46:10,631 | INFO     |: Disconnected from C0:83:0C:66:2F:3C


The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

In [32]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

print("ECHO TEST")
ble.send_command(CMD.ECHO, "Hello Artemis!")
print("Sent: 'Hello Artemis'")

# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print("Got back: ")
print(s)
print("\n\n")

print("GET_TIME_MILLIS TEST")

def handler(sender, time_stamp):
    time = time_stamp.decode("utf-8").split(":")[1]
    print("Time: "+time)

ble.start_notify(ble.uuid['RX_STRING'], handler)

ble.send_command(CMD.GET_TIME_MILLIS, "")


2023-02-04 18:13:33,164 | INFO     |: Already connected to a BLE device
ECHO TEST
Sent: 'Hello Artemis'
Time: )
Time: )
Got back: 
Robot says -> Hello Artemis! :)



GET_TIME_MILLIS TEST
Time: 1803043.0


In [36]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

print("TIME STAMP HANDLER TEST")

def handler(sender, time_stamp):
    time_stamp = time_stamp.decode("utf-8")
    time_stamp = time_stamp.split(":")[1]
    print(time_stamp)
    
ble.start_notify(ble.uuid['RX_STRING'], handler)

ble.send_command(CMD.GET_TIME_MILLIS, "")
ble.send_command(CMD.GET_TIME_MILLIS, "")
ble.send_command(CMD.GET_TIME_MILLIS, "")

2023-02-04 18:15:33,080 | INFO     |: Already connected to a BLE device
TIME STAMP HANDLER TEST
1923253.0
1923343.0
1923437.0


In [37]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

print("GET_TEMP_5S TEST")

temps_arr = []

def handler(sender, time_stamp_array):
    time_stamp_array = time_stamp_array.decode("utf-8")
    time_stamp_array = time_stamp_array.split("|")
    time = time_stamp_array[0].split(":")[1]
    temp = time_stamp_array[1].split(":")[1]
    temps_arr.append("Time: " + time + " | Temp: " + temp)
    print(temps_arr[len(temps_arr)-1])

ble.start_notify(ble.uuid['RX_STRING'], handler)

ble.send_command(CMD.GET_TEMP_5S, "")


2023-02-04 18:18:04,106 | INFO     |: Already connected to a BLE device
GET_TEMP_5S TEST
Time: 2074998.0 | Temp: 68.928
Time: 2075998.0 | Temp: 69.986
Time: 2076998.0 | Temp: 68.928
Time: 2077998.0 | Temp: 68.928
Time: 2078998.0 | Temp: 68.928
2023-02-04 18:27:15,796 | INFO     |: Disconnected from C0:83:0C:66:2F:3C


In [9]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

print("GET_TEMP_5S_RAPID TEST")

temps_arr = []

def handler(sender, time_stamp_array):
    time_stamp_array = time_stamp_array.decode("utf-8")
    time_stamp_array = time_stamp_array.split("|")
    time = time_stamp_array[0].split(":")[1]
    temp = time_stamp_array[1].split(":")[1]
    temps_arr.append("Time: " + time + " | Temp: " + temp)
    print(temps_arr[len(temps_arr)-1])

ble.start_notify(ble.uuid['RX_STRING'], handler)

ble.send_command(CMD.GET_TEMP_5S_RAPID, "")

2023-02-04 16:39:39,049 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:0c:66:2f:3c
2023-02-04 16:39:42,288 | INFO     |: Connected to c0:83:0c:66:2f:3c
GET_TEMP_5S_RAPID TEST
Time: 8232.0 | Temp: 78.446
Time: 8282.0 | Temp: 77.389
Time: 8332.0 | Temp: 78.446
Time: 8382.0 | Temp: 78.446
Time: 8432.0 | Temp: 78.446
Time: 8482.0 | Temp: 77.389
Time: 8532.0 | Temp: 77.389
Time: 8582.0 | Temp: 77.389
Time: 8632.0 | Temp: 76.331
Time: 8682.0 | Temp: 78.446
Time: 8732.0 | Temp: 77.389
Time: 8782.0 | Temp: 77.389
Time: 8832.0 | Temp: 78.446
Time: 8882.0 | Temp: 78.446
Time: 8932.0 | Temp: 77.389
Time: 8982.0 | Temp: 77.389
Time: 9032.0 | Temp: 77.389
Time: 9082.0 | Temp: 77.389
Time: 9132.0 | Temp: 77.389
Time: 9182.0 | Temp: 77.389
Time: 9232.0 | Temp: 78.446
Time: 9282.0 | Temp: 77.389
Time: 9332.0 | Temp: 78.446
Time: 9382.0 | Temp: 78.446
Time: 9432.0 | Temp: 77.389
Time: 9482.0 | Temp: 78.446
Time: 9532.0 | Temp: 78.446
Time: 9582.0 | Temp: 78.446
Time: 9632.0 | Temp: 77.

## Disconnect

In [6]:
# Disconnect
# ble.disconnect()